# 06: Go capabilities: Intro to concurrency

This section is a summary of https://go.dev/tour/

## Objectives
+ goroutines
+ channels 


## Summary

Go provides concurrency features as part of the core language.

This modules teaches you goroutines and channels, and how they are used to implement different concurrency patterns.

## Goroutines

A *goroutine* is a lightweight thread managed by the Go runtime.

It's started using the syntax:

```go
func f(x, y, z) { ... }

go f(x, y, z)
```

The evaluation of `f`, `x`, `y`, and `z` happens in the current goroutine and the execution of `f` happens in the new goroutine.

Goroutines run in the same address space, so access to shared memory must be synchronized.

The `sync` package provides useful primitives, although in practice you won't need to use them much.

### Lab

Write a program that defines a function `say(s string)` that sleeps for 100 millis and print `s` for 5 times.

Then in your main function do:

```go
go say("world")
say("hello")
```

Notice how the two words get alternated because they run in different threads.

| EXAMPLE: |
| :------- |
| See [01_goroutines-hello](01_goroutines-hello/) for a runnable example. |

## Channels

Channels are type conduit through which you can send and receive values with the channel operator `<-` (the data flows in the direction of the arrow).

```go
ch <- v     // Send v to channel ch
v := <- ch  // Receive from channel ch and assign to v
```

Like maps and slices, channels must be created before use:

```go
ch := make(chan int)
```

By default, send and receive operation are blocking operations until the other side is ready. This allows goroutines to synchronize without explicit locks or condition variables, and allows for a very simple approach to concurrency.

### Lab

Write a program that sums the numbers in a slice by distributing the work between two goroutines. Once both goroutines have completed their computation, it calculates the final result.

Hints:
+ Define a function `sum()` that receives a slice and a channel. The function should compute the sum of the slice elements and send the result to the received channel.
+ Note that the channel definition is `c chan type` with type identifying the type of elements that can be send and received in the channel.
+ In main(), define a int slide and then invoke `sum()` as two goroutines the get assigned half of the slice each.
+ Then, receive both results from the channel and print the result.

| EXAMPLE: |
| :------- |
| See [02_goroutines-channels](02_goroutines-channels/) for a runnable example. |

## Buffered channels

Channels can be buffered by providing a second argument to `make()` when initializing the channel that sets the buffer length:

```go
ch := make(chan int, 100)
```

In a buffered channel:
+ the send operation is blocking only when the buffer is full
+ the receive operation is blocking only when the buffer is empty

### Lab

Write a program that initializes an int buffered channel with length 2. Then, send two ints to the channel and then receive twice from it. What happens if you make an additional send? What happens if you make an additional receive?

| EXAMPLE: |
| :------- |
| See [03_goroutines-buffered-channels](03_goroutines-buffered-channels/) for a runnable example. |

## Range and Close

A sender can `close` a channel to indicate that no more values will be sent. Receivers can test whether a channel has been closed by assigning a second parameter to the receive expression:

```go
v, ok := <- ch
```

`ok` will be false if there are no more values to receive and the channel is closed.

The loop `for i := range c` will receive values from the channel repeatedly until it is closed.

| NOTE: |
| :---- |
| Only the sender should close a channel, never the receiver. Sending on a closed channel will cause a panic. |

Note also that channels aren't like files. As a result, you should only close the channel when you explicitly want to tell the receiver that no more values will be sent (e.g., when you want the receiver to terminate a `range` loop).

### Lab

Write a program that defines a function to calculate the fibonacci sequence as a go routine with the signature `fibonacci(n int, c chan int)`, where n is the number of elements to calculate. Then in the main, define a channel with capacity 10 and use the function as a goroutine. Receive the calculated elements with a `for` loop that ranges over the channel.

| EXAMPLE: |
| :------- |
| See [04_goroutines-range-and-close](04_goroutines-range-and-close/) for a runnable example. |

## Select

The `select` statement lets a goroutine wait on multiple communication operations.

A `select` blocks until one of its cases run, then it executes that case. It chooses one at random if multiple are ready:

```go
select {
  case val1 <- x:
    // ... do something
  case val2 <- y:
    // ... do some other thing
}
```

### Lab

Write a program that defines a function `fibonacci()` that receives two channels `c` and `quit`. Within the function, implement an infinite loop that includes a `select` statement. In the first case, of the `select` you use the first channel as an outbound channel in which you write the fibonacci sequence. In the second case, you break out of the infinite loop if you receive something on the second channel (`quit`).

HINT: you can use the syntax `<-quit` to detect when some information is available in the `quit` channel.

Then, in the main, initialize both channels as `int` channels. Then define an *anonymous goroutine* that writes the numbers 0 through 9 to the first channel and then sends a `0` to the `quit` channel.

```go
go func () {
  //... this is an anonymous goroutine
}()
```

Finally, invoke the `fibonacci()` sequence with both channels. Try to predict what is going to be the result before running it.

| EXAMPLE: |
| :------- |
| See [05_goroutines-select](05_goroutines-select/) for a runnable example. |

The `default` case in a `select` is run if no other case is ready.

You can use it to try a send or receive without blocking.

```go
select {
case i := <- c:   // blocks until something is ready on c  
  // ... use i in here, 
default:
  // executed if c is blocking
}
```

### Lab

Write a program that defines the following values:

```go
tick := time.Tick(100 * time.Millisecond)
boom := time.After(500 * time.Millisecond)
```

The first one would write to a channel `tick` every 100 millis, and the second will write something to a `boom` channel after 500 millis. That is, `time.Tick` and `time.After` return channels.

Then, use an infinite loop `for {...}` that wraps `select` that writes `"tick"` everytime you get a signal on `tick`, `"Boom"` when you get the signal on `boom` after 0.5 seconds, and use return to break out of the infinite loop; and a default case so that you can write `"..."` followed by a 50millis sleep when the other operations are blocked because both channels are empty.

| EXAMPLE: |
| :------- |
| See [06_goroutines-select-default](06_goroutines-select-default/) for a runnable example. |

### Lab

There can be many different binary trees with the same sequence of values stored in it.

For example, the following picture shows two binary trees storing the Fibonacci sequence: 1, 1, 2, 3, 5, 8, 13. Even when the shape of the tree is different, when you walk it, you'll get the same sequence:

![Tree](./images/tree.png)

In this lab, we'll write a function to check whether two binary trees store the same sequence using goroutines and channels.

In the lab, you'll use the `tree` package (`golang.org/x/tour/tree`), which defines the `tree.Tree` type as a struct:

```go
type Tree struct {
  Left *Tree
  value int
  Right *Tree
}
```

1. Implement the `Walk(t *tree.Tree, ch chan int)` function that walks the tree t sending all values from the tree to the channel ch.

2. Test the `Walk()` function. The function `tree.new(k)` constructs a randomly structured, but sorted, binary tree holding the values k, 2k, 3k, ...10k. In order to test it, create a new channel ch and trigger the walker using a goroutine `go Walk(tree.New(1), ch)`. Then read and print 10 values from the channel. Those should be the numbers from 1 to 10.

3. Implement a `Same(t1, t2 *tree.Tree) bool` the determines whether the trees t1 and t2 contain the same values. In the implementation, use the `Walk()` function.

4. Test the `Same()` function. `Same(tree.New(1), tree.New(1))` should return true while `Same(tree.New(1), tree.New(2))` should return false.

| EXAMPLE: |
| :------- |
| See [07_goroutines-equivalent-bin-trees](07_goroutines-equivalent-bin-trees/) for a runnable example. |

## Mutex

Channels are a great way for communicating among goroutines.

However, you might need other constructs if you just want to make sure that only one goroutine can access a variable at a time to avoid conflicts.

Go provides a construct for implementing *mutual exclusion* in your applications using a data structure conventionally called *mutex*.

The `sync.Mutex` structure provides two methods:
+ `Lock`
+ `Unlock`

You can define a block to be executed in mutual exclusion by surrounding it with a call to `Lock` and `Unlock`:

```go
func myFunc() {
  mu.Lock()
  // ... this block is executed in mutual exclusion
  mu.Unlock()
}
```

| NOTE: |
| :---- |
| `defer` keyword is a great way to ensure the mutex will be unlocked once the function has been executed.<br>Remember that `defer` statement schedules the execution of a function until the surrounding function returns. |

### Lab

Implement a counter `SafeCounter` that is safe to use concurrently by multiple goroutines.

Start by defining the `SafeCounter` struct which should include a mutex and a map that associates strings to ints.

Then, define the methods `Inc()` and `Value()` on `SafeCounter`. `Inc()` will receive a `key` and will increment the value of the corresponding key in the `SafeCounter` map in a concurrent-safe fashion. Similarly, `Value()` will receive a key and will return the existing value, also in a concurrent-safe fashion.

In `main()` initialize a `SafeCounter` value and then implement a loop that increments the key `someKey` 1000 times using go routines for each increment in the iteration.

Then, sleep your program for one second and use `Value()` to retrieve the value of the key.

| EXAMPLE: |
| :------- |
| See [08_mutex-safe-counter](08_mutex-safe-counter/) for a runnable example. |

### Lab

Use Go's concurrency features to parallelize a web crawler.

Consider the following program that gives you the scaffolding for a web crawler.

Modify the `Crawl` function to fetch URLs in parallel without fetching the same URL twice.

Hint: you can keep a cache of the URLs that have been fetches on a map, but maps alone are not safe for concurrent use, so you'll need to synchronize its access.

| EXAMPLE: |
| :------- |
| See [09_goroutines-web-crawler](09_goroutines-web-crawler/) for a runnable example, but please note that it doesn't work well (I need to practice more on concurrency topics). |